In [1]:
using Revise
using ReactiveMP, Rocket, GraphPPL, Random, LinearAlgebra, Plots, Flux, Zygote, ForwardDiff, DataFrames, DelimitedFiles, Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1423
┌ Warning: Package ReactiveMP does not have Flux in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Flux as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Flux into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function generate_based_on_normal(normal_sample)
    # if normal_sample > 0
    #     return rand(Bernoulli(0.99))
    # end
    # return rand(Bernoulli(0.01))
    return 1.0
end
normal_samples = rand(NormalMeanVariance(0, 1), 1000)
transformed = map(generate_based_on_normal, normal_samples);

In [3]:
function f(z)
    if z > 0
        return 0.99
    end
    return 0.01
end

@model function sensor_based_observation(n, n_iterations, n_samples, rng, learning_rate)

    # z = randomvar()
    x = randomvar()
    y = datavar(Float64, n)
    μ ~ NormalMeanVariance(0, 3)
    x ~ f(μ) where {meta = CVIApproximation(n_iterations, n_samples, rng, Descent(learning_rate))}
    
    for t in 1:n
        y[t] ~ Bernoulli(x)
    end

    return μ
end

constraints = @constraints begin
    q(x, μ) = q(μ)q(x)
end;

In [13]:
res = inference(
    model = Model(sensor_based_observation, length(transformed), 1000, 2000, nothing, 0.01),
    data = (y = transformed,),
    iterations = 2,
    free_energy = false,
    returnvars = (μ = KeepEach()),
    constraints = constraints,
    initmessages = (μ = NormalMeanVariance(10, 100),),
    initmarginals = (μ = NormalMeanVariance(10, 100),),
    showprogress = false,
)

Inference results:
-----------------------------------------
μ = NormalWeightedMeanPrecision{Float64}[NormalWeightedMeanPrecision{Float64}(xi=0.0...
x = ProdFinal{SampleList{(), Vector{Float64}, Vector{Float64}, ReactiveMP.SampleList...


In [11]:
res.posteriors[:μ]

2-element Vector{NormalWeightedMeanPrecision{Float64}}:
 NormalWeightedMeanPrecision{Float64}(xi=0.0, w=0.3333333333333333)
 NormalWeightedMeanPrecision{Float64}(xi=0.0, w=0.3333333333333333)